In [1]:
TRAIN_DF = "train_df.csv"

TEST_DF = "test_df.csv"
BOW_MODELS = "Models/BoW/" 
WE_MODELS = "Models/WE/" 
WORD_EMBEDDINGS = "../word_embeddings/"
PRE_METHOD='Raw' #'Lemmatized' for bow and we, 'Raw' for lm
#WE_MODEL="GLOVE_CC"
LM_MODEL='all-mpnet-base-v2'
FEATURE_TYPE="NN" #"BoW" #"LM" #"WE"
ROLLING_WINDOW_SIZE=10
SENT=False
ROUND_MAX=100

In [2]:
import pandas as pd
import numpy as np
from utilities import rolling_window, sa_features, save_embeddings
import optuna
from sentence_transformers import SentenceTransformer

import joblib
seed=23
np.random.seed(seed)

train_df = pd.read_csv(TRAIN_DF, sep='\t')
test_df = pd.read_csv(TEST_DF, sep='\t')
#train_df = rolling_window(train_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
#test_df = rolling_window(test_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
full_df = pd.concat([train_df,test_df])
####
full_df = rolling_window(full_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
####
"""
tokens_df = pd.read_csv("./tokens.csv", sep='\t')
total_df = tokens_df.merge(full_df.drop_duplicates(), on=['User','Post_Nr','Raw','Stemmed','Lemmatized','Label' ], 
                   how='left', indicator=True)
unused_df = total_df[total_df['_merge'] == 'left_only'].copy()
unused_df.drop(['_merge'], inplace=True, axis=1)
unused_df = rolling_window(unused_df,ROLLING_WINDOW_SIZE,1,PRE_METHOD)
unused_df = unused_df.query("Window_id < 90")
"""
"""
unused_df.to_csv("unused_df.csv",sep="\t")
lm=SentenceTransformer(LM_MODEL)
lm.max_seq_length=512 
unused_embeddings = lm.encode(unused_df['Text'],show_progress_bar=True,\
            output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)
save_embeddings("unused_embeddings.pkl", unused_embeddings)

if SENT:
    full_df = sa_features(full_df)
    unused_df = sa_features(unused_df)
full_df.to_pickle("full_df_sent.pkl")

full_df = pd.read_pickle("full_df_sent.pkl")
"""

#full_df.head()

'\nunused_df.to_csv("unused_df.csv",sep="\t")\nlm=SentenceTransformer(LM_MODEL)\nlm.max_seq_length=512 \nunused_embeddings = lm.encode(unused_df[\'Text\'],show_progress_bar=True,            output_value=\'sentence_embedding\', batch_size=64,convert_to_numpy=True)\nsave_embeddings("unused_embeddings.pkl", unused_embeddings)\n\nif SENT:\n    full_df = sa_features(full_df)\n    unused_df = sa_features(unused_df)\nfull_df.to_pickle("full_df_sent.pkl")\n\nfull_df = pd.read_pickle("full_df_sent.pkl")\n'

In [3]:
"""
### BoW FEATURES
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
nltk_stop_words = set(stopwords.words("english"))
import optuna
import joblib
TFIDF_STUDY_PATH="./BagOfWords/t1_tfidfvec_10"
MODEL_STUDY_PATH="./BagOfWords/t1_tuning_10.pkl"
tfidf_study = joblib.load(TFIDF_STUDY_PATH)
model_study = joblib.load(MODEL_STUDY_PATH)
tfidf_params = tfidf_study.best_trial.params
tfidf_params['stop_words'] = nltk_stop_words
tfidf_params['ngram_range'] = (1,tfidf_params['ngram_range'])
vectorizer = TfidfVectorizer(**tfidf_params)
model = SGDClassifier(**model_study.best_params)

"""

'\n### BoW FEATURES\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.linear_model import SGDClassifier\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom nltk.corpus import stopwords\nnltk_stop_words = set(stopwords.words("english"))\nimport optuna\nimport joblib\nTFIDF_STUDY_PATH="./BagOfWords/t1_tfidfvec_10"\nMODEL_STUDY_PATH="./BagOfWords/t1_tuning_10.pkl"\ntfidf_study = joblib.load(TFIDF_STUDY_PATH)\nmodel_study = joblib.load(MODEL_STUDY_PATH)\ntfidf_params = tfidf_study.best_trial.params\ntfidf_params[\'stop_words\'] = nltk_stop_words\ntfidf_params[\'ngram_range\'] = (1,tfidf_params[\'ngram_range\'])\nvectorizer = TfidfVectorizer(**tfidf_params)\nmodel = SGDClassifier(**model_study.best_params)\n\n'

In [4]:
"""
### WE FEATURES
from utilities import WordEmbeddingVectorizer
from sklearn.linear_model import SGDClassifier
import optuna
import joblib
STUDY_PATH="./Embeddings/t1_tuning_10.pkl"
study = joblib.load(STUDY_PATH)
model = SGDClassifier(**study.best_params)
we_vectorizer = WordEmbeddingVectorizer(WE_MODEL, WORD_EMBEDDINGS)
#full_df = full_df.sample(frac=1, random_state=seed).reset_index(drop=True) 
full_df['Vector'] = full_df['Text'].apply(lambda text: we_vectorizer.vectorize(text))
unused_df['Vector'] = unused_df['Text'].apply(lambda text: we_vectorizer.vectorize(text))

full_df
"""

'\n### WE FEATURES\nfrom utilities import WordEmbeddingVectorizer\nfrom sklearn.linear_model import SGDClassifier\nimport optuna\nimport joblib\nSTUDY_PATH="./Embeddings/t1_tuning_10.pkl"\nstudy = joblib.load(STUDY_PATH)\nmodel = SGDClassifier(**study.best_params)\nwe_vectorizer = WordEmbeddingVectorizer(WE_MODEL, WORD_EMBEDDINGS)\n#full_df = full_df.sample(frac=1, random_state=seed).reset_index(drop=True) \nfull_df[\'Vector\'] = full_df[\'Text\'].apply(lambda text: we_vectorizer.vectorize(text))\nunused_df[\'Vector\'] = unused_df[\'Text\'].apply(lambda text: we_vectorizer.vectorize(text))\n\nfull_df\n'

In [5]:
"""
### LM FEATURES
from sklearn.linear_model import SGDClassifier
import optuna
import joblib
from utilities import load_embeddings
LM_MODEL='all-mpnet-base-v2'
STUDY_PATH = "./LanguageModels/t1_tuning_10.pkl"
study = joblib.load(STUDY_PATH)
study.best_params
model = SGDClassifier(**study.best_params)
#train_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
#val_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
#unused_embeddings = load_embeddings(f"./unused_embeddings.pkl")

#full_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)
#full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
full_df = pd.read_pickle("full_df_pkl.pkl")

unused_df = pd.read_pickle("final_unused_lm.pkl")
#unused_df['Vector'] = pd.DataFrame(data=unused_embeddings).values.tolist()
"""

'\n### LM FEATURES\nfrom sklearn.linear_model import SGDClassifier\nimport optuna\nimport joblib\nfrom utilities import load_embeddings\nLM_MODEL=\'all-mpnet-base-v2\'\nSTUDY_PATH = "./LanguageModels/t1_tuning_10.pkl"\nstudy = joblib.load(STUDY_PATH)\nstudy.best_params\nmodel = SGDClassifier(**study.best_params)\n#train_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")\n#val_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")\n#unused_embeddings = load_embeddings(f"./unused_embeddings.pkl")\n\n#full_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)\n#full_df[\'Vector\'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()\nfull_df = pd.read_pickle("full_df_pkl.pkl")\n\nunused_df = pd.read_pickle("final_unused_lm.pkl")\n#unused_df[\'Vector\'] = pd.DataFrame(data=unused_embeddings).values.tolis

In [6]:
"""
"""
### LM FEATURES FOR NN
from sklearn.linear_model import SGDClassifier
import torch
import optuna
import joblib
from utilities import load_embeddings, LmNeuralNetwork, WritingWindowDataset
STUDY_PATH = "./LanguageModels/NN_hyperparameterstudy10.pkl"
study = joblib.load(STUDY_PATH)
study.best_params
params = optuna.trial.FixedTrial(study.best_trial.params)
train_sentence_embeddings = load_embeddings(f"./LanguageModels/train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
val_sentence_embeddings = load_embeddings(f"./LanguageModels/val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")

full_sentence_embeddings = np.concatenate((train_sentence_embeddings,val_sentence_embeddings), axis=0)#full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
full_df['Vector'] = pd.DataFrame(data=full_sentence_embeddings).values.tolist()
unused_df = pd.read_pickle("final_unused_lm.pkl")

train_ds = WritingWindowDataset(full_sentence_embeddings, full_df['Label'])




In [7]:
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import f1_score
#from torchmetrics.functional import f1_score
BATCH_SIZE = 32
epochs=10

def train_nn(trainloader, study):
    params = optuna.trial.FixedTrial(study.best_trial.params)
    model= LmNeuralNetwork(params)

    model = model.cuda()
    optimizer = AdamW(model.parameters(), lr=study.best_trial.params['learning_rate'])#, lr= learning_rate)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()

            

    for epoch_num in range(epochs):
        
        total_loss_train = 0
        train_preds = np.array([])
        train_targets = np.array([])
        model.train()
        for train_input, train_label in tqdm(trainloader):

            train_label = train_label.to(device)
            train_label = train_label.float()
            train_label = train_label.unsqueeze(1)
            features = train_input.to(device)
            

            output = model(features)
            batch_loss = criterion(output, train_label)
            total_loss_train += float(batch_loss.item())
            
            train_preds = np.concatenate((train_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
            train_targets = np.concatenate((train_targets,train_label.detach().cpu().numpy().flatten()),axis=0)

            
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        train_loss=total_loss_train/len(train_preds)
        train_f1 = f1_score(train_targets,train_preds)
            
        print(
                f'Epochs: {epoch_num + 1} | Train Loss: {train_loss: .4f} \
                | Train F1: {train_f1: .4f}')

        
    return model

In [8]:

torch.manual_seed(23)


In [9]:
from sklearn.model_selection import StratifiedKFold

def nn_cv_classification(full_df, study,filename):
    skf = StratifiedKFold(n_splits=5)
    user_label_df =full_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
            train_users = [users[f] for f in train_index]
            test_users = [users[f] for f in test_index]
            train_ids = train_df.index[train_df['User'].isin(train_users)].to_list()
            train_folds = full_df[full_df['User'].isin(train_users)].copy()
            train_folds = train_folds.sample(frac=1, random_state=seed).reset_index(drop=True) 

            test_folds = full_df[full_df['User'].isin(test_users)].copy()
            test_folds = pd.concat([test_folds,unused_df])
            train_data = torch.utils.data.Subset(train_ds, train_ids)
            
            trainloader = torch.utils.data.DataLoader(
                            train_data, 
                            batch_size=32, 
                            shuffle=True)
            model = train_nn(trainloader, study)
            model.eval()
            print(f"model for fold {fold} trained")
            true_label=test_folds[['User','Label']].groupby('User').max().to_dict()['Label']
            true_labels = {subject:true_label[subject] for subject in true_label}
            with open(f"{filename}_{fold}.txt", "w") as f:
                for user in true_labels:
                    f.write(f"({user},{true_labels[user]})")
                    for window_id in range(ROUND_MAX-ROLLING_WINDOW_SIZE+1):
                        test_f = test_folds.query("User==@user and Window_id==@window_id")
                        if len(test_f)>0:
                            f.write(" ")
                            
                            X_test = test_f['Vector'].values.tolist()[0]
                            with torch.no_grad():
                                X_test = torch.tensor([X_test]).cuda() #added

                                
                                pred = model(X_test)[0]
                                f.write(f"({round(pred.item())},{pred.item()})")
                    f.write("\n")

In [10]:
nn_cv_classification(full_df,study, "NN")


100%|██████████| 2272/2272 [00:26<00:00, 86.85it/s] 


Epochs: 1 | Train Loss:  0.0220                 | Train F1:  0.6431


100%|██████████| 2272/2272 [00:07<00:00, 309.15it/s]


Epochs: 2 | Train Loss:  0.0210                 | Train F1:  0.6431


100%|██████████| 2272/2272 [00:07<00:00, 315.13it/s]


...

Epochs: 8 | Train Loss:  0.0114                 | Train F1:  0.8290


100%|██████████| 3059/3059 [00:09<00:00, 311.68it/s]


Epochs: 9 | Train Loss:  0.0111                 | Train F1:  0.8330


100%|██████████| 3059/3059 [00:09<00:00, 310.96it/s]


Epochs: 10 | Train Loss:  0.0109                 | Train F1:  0.8372
model for fold 4 trained


In [7]:
from sklearn.model_selection import StratifiedKFold

def cv_classification(full_df, model,filename):
    skf = StratifiedKFold(n_splits=5)
    user_label_df =full_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
            if fold!=0:
                break
            train_users = [users[f] for f in train_index]
            test_users = [users[f] for f in test_index]

            train_folds = full_df[full_df['User'].isin(train_users)].copy()
            train_folds = train_folds.sample(frac=1, random_state=seed).reset_index(drop=True) 

            test_folds = full_df[full_df['User'].isin(test_users)].copy()
            test_folds = pd.concat([test_folds,unused_df])

            if FEATURE_TYPE=="BoW":
                X_train = vectorizer.fit_transform(train_folds['Text'])
            else:
                X_train = pd.DataFrame(train_folds['Vector'].values.tolist(), index = train_folds.index)
            y_train = train_folds['Label']
            if SENT:
                X_train = np.c_[X_train,train_folds['polarity'],train_folds['subjectivity'],train_folds['negativity'],train_folds['positivity'],train_folds['neutrality'], train_folds['compound']] 

            model.fit(X_train, y_train)
            print(f"model for fold {fold} trained")
            true_label=test_folds[['User','Label']].groupby('User').max().to_dict()['Label']
            true_labels = {subject:true_label[subject] for subject in true_label}
            with open(f"{filename}_{fold}.txt", "w") as f:
                for user in true_labels:
                    f.write(f"({user},{true_labels[user]})")
                    for window_id in range(ROUND_MAX-ROLLING_WINDOW_SIZE+1):
                        test_f = test_folds.query("User==@user and Window_id==@window_id")
                        if len(test_f)>0:
                            f.write(" ")
                            if FEATURE_TYPE=="BoW":
                                X_test = vectorizer.transform(test_f['Text'])
                                if SENT:
                                    X_test = np.r_[X_test,test_f['polarity'],test_f['subjectivity'],test_f['negativity'],test_f['positivity'],test_f['neutrality'], test_f['compound']] 
                                X_test.reshape(1,-1)
                            else:
                                X_test = test_f['Vector'].values.tolist()[0]
                                if SENT:
                                    X_test = np.r_[X_test,test_f['polarity'],test_f['subjectivity'],test_f['negativity'],test_f['positivity'],test_f['neutrality'], test_f['compound']] 
                                X_test = [X_test] #added

                            
                            pred = model.predict(X_test)[0]
                            prob = model.predict_proba(X_test)[0].tolist()[1]
                            f.write(f"({pred},{prob})")
                    f.write("\n")

In [8]:
#cv_classification(full_df, model, FEATURE_TYPE)


model for fold 0 trained
